#**TODO**

Refine keywords : strip \n, ", ' etc ...

Handle ill-encoded keywords : utf-8 ...?

#**Import Modules**

In [0]:
import openpyxl

In [0]:
!pip install xlsxwriter
import xlsxwriter

     |████████████████████████████████| 143kB 2.8MB/s 


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
import gc

In [0]:
from os import listdir
from os.path import isfile, join

In [0]:
from google.colab import files

#**Mount Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/My Drive/Colab Notebooks/Paper/Trade

/content/drive/My Drive/Colab Notebooks/Paper/Trade


In [0]:
ls

Crawler/  Data/  Paper-Authors/  Paper-Keywords/


#**Read and Process Data**

In [0]:
def pair_to_vector(idx, length):
    res = [0] * length
    for i in idx:
        res[i] = 1
    return res

In [0]:
DATAPATH = "./Data/raw/Korean/"

In [0]:
# fetch journal names
journals = [f for f in listdir(DATAPATH) if isfile(join(DATAPATH, f))]

for i in range(len(journals)):
    journals[i] = journals[i][:-5]

journals

['journal_of_korea_trade',
 'journal_of_international_trade_and_industry_studies',
 'korea_trade_review']

In [0]:
keywords = []
papers = []
keyword_paper_pair = []
curr_idx = -1

for i in range(len(journals)):
    # open workbook
    wb = openpyxl.load_workbook(DATAPATH + journals[i] + ".xlsx")
    ws = wb.active
    MAXROW = ws.max_row
    MAXCOL = ws.max_column
    curr = 0

    for r in ws.rows:
        # ignore header
        if(r[0].row == 1):
            continue
        # halt loop
        if(r[0].row > MAXROW):
            break
        # update cursor
        if(isinstance(r[2].value, str)):
            curr = r[2].value
            papers.append(curr)
            curr_idx += 1

        # row corresponds to a keyword
        if(isinstance(r[3].value, str)):
            # duplicate exists
            if(r[3].value.lower() in keywords):
                keyword_paper_pair[keywords.index(r[3].value.lower())].append(curr_idx)
            # duplicate doesn't exist
            else:
                keywords.append(r[3].value.lower())
                keyword_paper_pair.append([curr_idx])
    
# encode keyword-paper pair to vector
keyword_vector = []
for i in range(len(keyword_paper_pair)):
    keyword_vector.append(pair_to_vector(keyword_paper_pair[i], curr_idx + 1))

wb.close()

In [0]:
len(papers)

1797

In [0]:
len(keywords)

5428

In [0]:
len(keyword_vector)

5428

#**Write Paper-Keyword Matrix to Excel**

In [0]:
def initial(s):
    res = ''
    for a in s.split(' '):
        res += a[0]
    return res

In [0]:
TARGETPATH = "./Data/mat/Korean/"

In [0]:
RESFILENAME = ""
for journal in journals:
    RESFILENAME += "_" + initial(journal.replace("_", " ").upper())
RESFILENAME = RESFILENAME[1:]

wb = xlsxwriter.Workbook(TARGETPATH + RESFILENAME + "_mat.xlsx")
ws = wb.add_worksheet()

for i in range(len(papers)):
    ws.write(i + 1, 0, papers[i])
for j in range(len(keyword_vector)):
    ws.write(0, j + 1, keywords[j])
    for i in range(len(keyword_vector[j])):
        ws.write(i + 1, j + 1, keyword_vector[j][i])

wb.close()

#**Download Files**

In [0]:
files.download(TARGETPATH + RESFILENAME + "_mat.xlsx")